In [1]:
# Define the API settings
CLIENT_SECRETS_FILE = "client_secret.json"

SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

# Import necessary modules, mostly google apiclient and
# google authentication related modules
import os
import pickle
import google.oauth2.credentials
 
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
 
# function that authenticates with the credentials related
# to Google APIs and returns a service that can be used to
# interact with YouTube API

def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [2]:
# Get the service
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
youtube = get_authenticated_service()

FileNotFoundError: [Errno 2] No such file or directory: 'client_secret.json'

In [ ]:
data_dict = dict()
i = 0
video_id = "xWrtgnCTmj4" 
#video_title = item['snippet']['title']
video_title = "Trump delivers remarks in Pennsylvania"

first_page = True
page_token = None
nextPageExist = True
while nextPageExist:
    if first_page:
        comment_threads = youtube.commentThreads().list(part="snippet",
                                                        maxResults = 50,
                                                        videoId = video_id).execute()
        first_page = False

    else:
        comment_threads = youtube.commentThreads().list(part="snippet",
                                                        maxResults = 50,
                                                        pageToken = page_token,
                                                        videoId = video_id).execute()

    for comment_thread in comment_threads['items']:
        comment = comment_thread['snippet']['topLevelComment']['snippet']['textDisplay']
        comment_id = comment_thread['snippet']['topLevelComment']['id']
        like_count = comment_thread['snippet']['topLevelComment']['snippet']['likeCount']
        publish_date = comment_thread['snippet']['topLevelComment']['snippet']['publishedAt']
        data_dict[i] = {'video_id':video_id,
                        #'video_title':video_title,
                        'comment_id':comment_id,
                        'comment_date':publish_date,
                        'comment':comment,
                        'like_count':like_count}
        i += 1

    if 'nextPageToken' in comment_threads:
        nextPageExist = True
        page_token = comment_threads['nextPageToken']
    else:
        nextPageExist = False